### Notebook for Openroute service testings

In [1]:
import openrouteservice as ors
import folium 

In [7]:
# Note you have to create an account and get your own personal API key
client = ors.Client(key = "") 

In [11]:
_map = folium.Map(location=[-37.8376, 144.9631], tiles="Cartodb Positron", zoom_start=13)

coords = [[144.9671, -37.8183], [144.9610, -37.7983]]

route = client.directions(coordinates = coords,
                          profile = 'driving-car', 
                          format = 'geojson')

folium.PolyLine(locations = [list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']], color = 'blue').add_to(_map)

In [12]:
# Save the map file (open route services)

_map.save("../plots/flinders_to_Uni_map.html")
_map